In [1]:
pip install cdqa

     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 102kB 4.8MB/s 
     |████████████████████████████████| 286kB 40.5MB/s 
     |████████████████████████████████| 10.5MB 38.7MB/s 
     |████████████████████████████████| 317kB 54.0MB/s 
     |████████████████████████████████| 6.7MB 42.5MB/s 
     |████████████████████████████████| 748.9MB 19kB/s 
     |████████████████████████████████| 92kB 7.2MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 1.0MB 29.8MB/s 
     |████████████████████████████████| 890kB 46.3MB/s 
  Created wheel for cdqa: filename=cdqa-1.3.9-cp36-none-any.whl size=47640 sha256=b880630bcdd2a6be45c1d51fdd99c811fdbe5cd4e345af807c646ca8e4668ed3
  Stored in directory: /root/.cache/pip/wheels/8b/9a/68/d3f7651ea29c30d1bebc9e946bf5a8cf922e1c86fb6b8a33d9
  Created wheel for tika: filename=tika-1.19-cp36-none-any.whl size=29222 sha256=2c4818ee2a0876905d9456515eac597a3f212dce89fade3ed5ea319

In [1]:
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.utils.download import download_model, download_bnpp_data
from cdqa.pipeline.cdqa_sklearn import QAPipeline


/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [0]:
from nltk import sent_tokenize, word_tokenize

In [0]:
basedir = '/content/test'

In [0]:
import os

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
files=pd.DataFrame()
files['title'] = os.listdir(basedir)
files['paths']=' '
files['paths'] = [os.path.join(basedir, name) for name in files['title']]
files['paragraphs']=' '
files['paragraphs']=[open((i), 'r', encoding="utf8", errors='ignore').read().strip() for i in files['paths'] ]

In [7]:
files.head()

,title,paths,paragraphs
0,Nineteen_data.txt,/content/test/Nineteen_data.txt,hh{{Государственный деятель\n |имя ...
1,Seventeen_data.txt,/content/test/Seventeen_data.txt,hh{{Персона}}\n'''Абид''' (род. ок. 1590г; раб...
2,Eighteen_data.txt,/content/test/Eighteen_data.txt,hh{{однофамильцы|Абильгор}}\n{{Художник\n |имя...
3,Twenty_data.txt,/content/test/Twenty_data.txt,hh{{Художник\n |имя = Вильмош...


In [0]:
files['paragraphs']=files['paragraphs'].apply(lambda x:sent_tokenize(x) )

In [0]:
files.to_csv('new_dataset.csv')

In [0]:
df=pd.read_csv('new_dataset.csv',converters={'paragraphs': literal_eval})

In [0]:
new_df=df.head(n=3)

In [12]:
#cdqa_pipeline = QAPipeline(reader='C:\\Users\\Алматы\\cdQA\\cdqa\\reader\\bert_qa_vCPU-sklearn.joblib')
from cdqa.pipeline import QAPipeline

import wget
url = "https://github.com/cdqa-suite/cdQA/releases/download/bert_qa/bert_qa.joblib"
wget.download(url, 'bert_qa.joblib')

'bert_qa.joblib'

In [13]:
cdqa_pipeline = QAPipeline(reader='bert_qa.joblib')
cdqa_pipeline.fit_retriever(df=df)

100%|██████████| 231508/231508 [00:00<00:00, 4379405.52B/s]


QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po...size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [0]:
query = 'В каком году родился Иван Шаповалов?'
prediction = cdqa_pipeline.predict(query)

In [24]:
prediction

('в 1817',
 'Nineteen_data.txt',
 '==Биография==\n\nИван Шаповалов родился в 1817 году в семье крепостного крестьянина села Николаевки, [[Кременчугский уезд|Кременчугского уезда]], [[Полтавская губерния|Полтавской губернии]].',
 5.053192349440856)

In [25]:
print('query: {}\n'.format(query))
print('answer: {}\n'.format(prediction[0]))
print('title: {}\n'.format(prediction[1]))
print('paragraph: {}\n'.format(prediction[2]))

query: В каком году родился Иван Шаповалов?

answer: в 1817

title: Nineteen_data.txt

paragraph: ==Биография==

Иван Шаповалов родился в 1817 году в семье крепостного крестьянина села Николаевки, [[Кременчугский уезд|Кременчугского уезда]], [[Полтавская губерния|Полтавской губернии]].



In [65]:
query = 'Саврасов учился в?'
prediction = cdqa_pipeline.predict(query)
prediction

('Алексей Кондратьевич',
 'Nineteen_data.txt',
 'М.\xa0Прянишникова]], [[Саврасов, Алексей Кондратьевич|А.',
 0.7530701209073714)

In [66]:
print('query: {}\n'.format(query))
print('answer: {}\n'.format(prediction[0]))
print('title: {}\n'.format(prediction[1]))
print('paragraph: {}\n'.format(prediction[2]))

query: Саврасов учился в?

answer: Алексей Кондратьевич

title: Nineteen_data.txt

paragraph: М. Прянишникова]], [[Саврасов, Алексей Кондратьевич|А.

